<a href="https://colab.research.google.com/github/MrckHckr/artemis2-go-tests/blob/main/notebooks/09_live_go_nogo_dashboardipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Core Stage RS-25

In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# =====================
# DATI SIMULATI
# =====================
np.random.seed(42)
time = np.arange(0, 300)  # secondi

pressure = 5 + np.random.normal(0, 0.25, len(time))
temperature = 850 + np.random.normal(0, 15, len(time))

# Anomalie artificiali
pressure[180] += 1.2
temperature[220] += 90

df = pd.DataFrame({
    "Time": time,
    "Pressure": pressure,
    "Temperature": temperature
})

# =====================
# SOGLIE OPERATIVE
# =====================
P_MIN, P_MAX = 4.5, 5.5
T_MIN, T_MAX = 800, 900

pressure_go = df["Pressure"].between(P_MIN, P_MAX).all()
temperature_go = df["Temperature"].between(T_MIN, T_MAX).all()

# =====================
# DECISIONE FINALE
# =====================
if pressure_go and temperature_go:
    final_status = "GO"
    final_value = 0.2
elif pressure_go or temperature_go:
    final_status = "WARNING"
    final_value = 0.6
else:
    final_status = "NO-GO"
    final_value = 0.95

# =====================
# DASHBOARD
# =====================
fig = make_subplots(
    rows=3, cols=2,
    specs=[
        [{"type": "xy", "colspan": 2}, None],
        [{"type": "xy"}, {"type": "indicator"}],
        [{"type": "indicator", "colspan": 2}, None]
    ],
    subplot_titles=[
        "Hot Fire Test – Pressione",
        "Temperatura Core Stage",
        "Decisione Pressione",
        "Decisione Temperatura",
        "Semaforo Complessivo RS-25"
    ]
)

# ---- Pressione
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Pressure"], name="Pressure (bar)", line=dict(width=2)),
    row=1, col=1
)
fig.add_hrect(y0=P_MIN, y1=P_MAX, fillcolor="green", opacity=0.15, row=1, col=1)

# ---- Temperatura
fig.add_trace(
    go.Scatter(x=df["Time"], y=df["Temperature"], name="Temperature (K)", line=dict(color="orange", width=2)),
    row=2, col=1
)
fig.add_hrect(y0=T_MIN, y1=T_MAX, fillcolor="green", opacity=0.15, row=2, col=1)

# ---- Gauge Pressione
fig.add_trace(
    go.Indicator(
        mode="gauge",
        value=0 if pressure_go else 1,
        gauge={
            "axis": {"range": [0, 1]},
            "steps": [
                {"range": [0, 0.5], "color": "green"},
                {"range": [0.5, 1], "color": "red"}
            ]
        }
    ),
    row=2, col=2
)

# ---- Gauge Temperatura
fig.add_trace(
    go.Indicator(
        mode="gauge",
        value=0 if temperature_go else 1,
        gauge={
            "axis": {"range": [0, 1]},
            "steps": [
                {"range": [0, 0.5], "color": "green"},
                {"range": [0.5, 1], "color": "red"}
            ]
        }
    ),
    row=3, col=1
)

# ---- Semaforo finale
fig.add_trace(
    go.Indicator(
        mode="gauge",
        value=final_value,
        gauge={
            "axis": {"range": [0, 1]},
            "steps": [
                {"range": [0, 0.4], "color": "green"},
                {"range": [0.4, 0.8], "color": "yellow"},
                {"range": [0.8, 1], "color": "red"}
            ],
            "threshold": {
                "line": {"color": "black", "width": 4},
                "value": final_value
            }
        }
    ),
    row=3, col=1
)

# ---- FINAL DECISION come oggetto separato in basso
fig.add_annotation(
    text=f"<b>FINAL DECISION: {final_status}</b>",
    x=0.5,
    y=-0.05,   # sotto tutto il grafico
    xref="paper",
    yref="paper",
    showarrow=False,
    font=dict(size=28, color="black")
)

# Layout generale
fig.update_layout(
    height=1000,
    title="🚀 Core Stage RS-25 – Integrated Test Dashboard",
    showlegend=False
)

fig.show()